<a href="https://colab.research.google.com/github/yusrilandrian/TugasPraktikumML_167-269/blob/yusril/MODUL_2_KEGIATAN_2_(Preprocessing).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LOAD DATA**

In [40]:
import pandas as pd
import numpy as np
import os, zipfile, cv2, shutil, math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm as tq
from tensorflow.keras.optimizers import Adam
from keras import optimizers
from keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Conv2D, MaxPool2D, Flatten, Dropout
from google.colab import drive

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
# masuk ke directory penyimpanan dataset anda
%cd /content/drive/MyDrive/kaggle

/content/drive/MyDrive/kaggle


In [43]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/kaggle"

In [44]:
ls

 breakhis-400x.zip                     SOB_B_A-14-22549AB-400-007.png
 dataset/                              SOB_B_A-14-22549AB-400-022.png
 kaggle.json                          'SOB_B_A-14-22549CD-400-003 (1).png'
 orchids.zip                           SOB_B_A-14-22549CD-400-003.png
 SOB_B_A-14-22549AB-400-001.png        SOB_B_A-14-22549G-400-018.png
'SOB_B_A-14-22549AB-400-007 (1).png'   SOB_B_F-14-21998EF-400-004.png
'SOB_B_A-14-22549AB-400-007 (2).png'   SOB_M_DC-14-11031-400-005.png
'SOB_B_A-14-22549AB-400-007 (3).png'


In [45]:
local_zip = '/content/drive/MyDrive/kaggle/orchids.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/drive/MyDrive/Kaggle')
zip_ref.close()

In [ ]:
!kaggle datasets download -d mikful/orchids

401 - Unauthorized


In [46]:
os.listdir('/content/drive/MyDrive/kaggle/dataset')

['Angraecum',
 'Brassavola',
 'Brassia',
 'Cattleya',
 'Cymbidium',
 'Dendrobium',
 'Encyclia',
 'Epidendrum',
 'Lycaste',
 'Masdevallia',
 'Maxillaria',
 'Miltonia',
 'Miltoniopsis',
 'Odontoglossum',
 'Oncidium',
 'Paphiopedilum',
 'Phalaenopsis',
 'Vanda',
 'Vanilla',
 'Zygopetalum',
 'dataset']

In [ ]:
# Path Directory
brassia_dir = '/content/dataset/Brassia'
miltonia_dir = '/content/dataset/Miltonia'

In [47]:
# Pastikan Folder hanya berisi jpg
from PIL import Image
base_dataset = '/content/drive/MyDrive/kaggle/dataset'
class_dir = ['Brassia','Miltonia']
for class_item in class_dir:
  cur_dir = base_dataset + '/' + class_item + '/'
  dataset = os.listdir(cur_dir)
  for item in dataset :
    im = Image.open(f"{cur_dir}{item}")
    if item.endswith('.jpg'):
      im.save(f"{cur_dir}{item.split('.')[0]}.png", "png")
    else :
      im.save(f"{cur_dir}{item.split('.')[0]}.png", "png")
  for item in dataset :
    if not item.endswith('.png'):
      os.remove(os.path.join(cur_dir, item))

In [48]:
file_name = []
tag = []
full_path = []
for path, subdirs, files in os.walk(base_dataset):
    for name in files:
        full_path.append(os.path.join(path, name)) 
        tag.append(path.split('/')[-1])        
        file_name.append(name)

In [49]:
# membuat dataframe dari variabel baru
data = pd.DataFrame({'path':full_path,'file_name':file_name,'tag':tag})
print(data.groupby(['tag']).size())

tag
Angraecum         604
Brassavola       1062
Brassia           625
Cattleya          614
Cymbidium         572
Dendrobium        540
Encyclia          650
Epidendrum        632
Lycaste           612
Masdevallia       690
Maxillaria        440
Miltonia          462
Miltoniopsis      622
Odontoglossum     438
Oncidium          472
Paphiopedilum     642
Phalaenopsis      928
Vanda             518
Vanilla           430
Zygopetalum       574
dtype: int64


In [50]:
x = data['path']
y = data['tag']

In [51]:
# Splitting
!pip install split_folders

In [52]:
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.30)

In [54]:
# menyatukan ke dalam masing-masing dataframe
data_tr = pd.DataFrame({'path':X_train, 'tag':y_train, 'set':'training'})

data_val = pd.DataFrame({'path':X_val, 'tag':y_val, 'set':'validation'})

In [55]:
print('train size', len(data_tr))
print('val size', len(data_val))

train size 8488
val size 3639


In [62]:
dataset_path = '/content/drive/MyDrive/kaggle/model_dataset'

data_all = data_tr.append([data_tr, data_val]).reset_index(drop=1)

for index, row in tq(data_all.iterrows()):
    
    #detect filepath
    file_path = row['path']
    if os.path.exists(file_path) == False:
            file_path = os.path.join(datasource_path,row['tag'],row['image'].split('.')[0])            
    
    #make folder destination dirs
    if os.path.exists(os.path.join(dataset_path,row['set'],row['tag'])) == False:
        os.makedirs(os.path.join(dataset_path,row['set'],row['tag']))
    
    #define file dest
    destination_file_name = file_path.split('/')[-1]
    file_dest = os.path.join(dataset_path,row['set'],row['tag'],destination_file_name)
    
    #copy file from source to dest
    if os.path.exists(file_dest) == False:
        shutil.copy2(file_path,file_dest)

0it [00:00, ?it/s]

In [63]:
os.listdir('/content/drive/MyDrive/kaggle/model_dataset/training')

['Paphiopedilum',
 'Lycaste',
 'Masdevallia',
 'Vanilla',
 'Dendrobium',
 'Oncidium',
 'Phalaenopsis',
 'Brassavola',
 'Brassia',
 'Vanda',
 'Encyclia',
 'Cymbidium',
 'Epidendrum',
 'Zygopetalum',
 'Angraecum',
 'Miltonia',
 'Cattleya',
 'Maxillaria',
 'Odontoglossum',
 'Miltoniopsis']

In [64]:
brassia_train_dir = '/content/dataset/model_dataset/train/Brassia'
brassia_val_dir = '/content/dataset/model_dataset/val/Brassia'
miltonia_train_dir = '/content/dataset/model_dataset/train/Miltonia'
miltonia_val_dir = '/content/dataset/model_dataset/val/Miltonia'

# **PREPROCESSING**

In [68]:
train_dir = '/content/dataset/model_dataset/train'
val_dir = '/content/dataset/model_dataset/val'

In [70]:
# Gather data train
train_data = []
train_label = []

train_dir='/content/drive/MyDrive/kaggle/model_dataset/training'
for r, d, f in os.walk(train_dir):
    for file in f:
        if ".png" in file:
            imagePath = os.path.join(r, file)
            image = cv2.imread(imagePath)
            image = cv2.resize(image, (250,250))
            train_data.append(image)
            label = imagePath.split(os.path.sep)[-2]
            train_label.append(label)

train_data = np.array(train_data)
train_label = np.array(train_label)

In [71]:
# Gather data validation
val_data = []
val_label = []

vali_dir='/content/drive/MyDrive/kaggle/model_dataset/validation'
for r, d, f in os.walk(vali_dir):
    for file in f:
        if ".png" in file:
            imagePath = os.path.join(r, file)
            image = cv2.imread(imagePath)
            image = cv2.resize(image, (250,250))
            val_data.append(image)
            label = imagePath.split(os.path.sep)[-2]
            val_label.append(label)

val_data = np.array(val_data)
val_label = np.array(val_label)

In [72]:
# Tampilkan shape dari data train dan data validation
print('Data Training Shape : ', train_data.shape)
print('Data Validation Shape : ', val_data.shape)

Data Training Shape :  (485, 250, 250, 3)
Data Validation Shape :  (232, 250, 250, 3)


In [73]:
# Normalisasi dataset
print("Data sebelum di-normalisasi ", train_data[0][0][0])

x_train = train_data.astype('float32') / 255.0
x_val = val_data.astype('float32') / 255.0
print("Data setelah di-normalisasi ", x_train[0][0][0])

Data sebelum di-normalisasi  [255 255 255]
Data setelah di-normalisasi  [1. 1. 1.]


In [76]:
# Transformasi label encoder
from sklearn.preprocessing import LabelEncoder

print("Label sebelum di-encoder ", train_label[0:10])

lb = LabelEncoder()
y_train = lb.fit_transform(train_label)
y_test = lb.fit_transform(val_label)

print("Label setelah di-encoder ", y_train[0:10])

Label sebelum di-encoder  ['Paphiopedilum' 'Paphiopedilum' 'Paphiopedilum' 'Paphiopedilum'
 'Paphiopedilum' 'Paphiopedilum' 'Paphiopedilum' 'Paphiopedilum'
 'Paphiopedilum' 'Lycaste']
Label setelah di-encoder  [15 15 15 15 15 15 15 15 15  8]


# **DEFINISIKAN MODEL**

In [77]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Conv2D, MaxPool2D, Flatten, GlobalMaxPooling2D, GlobalAveragePooling2D, AveragePooling2D

MODEL MAX POOLING

In [79]:
# Feature Extraction Layer
model = Sequential()

model.add(InputLayer(input_shape=[250,250,3]))
model.add(Conv2D(filters=16, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, padding='same'))
model.add(Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, padding='same'))
model.add(Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(GlobalMaxPooling2D())
model.add(Flatten())

# Fully Connected Layer
model.add(Dense(128, activation='relu'))
model.add(Dense(9, activation='softmax'))

# Print model summary
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 250, 250, 16)      448       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 125, 125, 16)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 125, 125, 32)      4640      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 63, 63, 64)        18496     
_________________________________________________________________
global_max_pooling2d_1 (Glob (None, 64)                0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 64)               

In [85]:
# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='sparse_categorical_crossentropy', 
              metrics=['acc'])

MODEL AVERAGE POOLING

In [80]:

# Scenario Modelling 2

# Feature Exctraction Layer
Model = Sequential()

Model.add(InputLayer(input_shape=[250,250,3]))
Model.add(Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'))
Model.add(AveragePooling2D(pool_size=2, padding='same'))
Model.add(Conv2D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu'))
Model.add(AveragePooling2D(pool_size=2, padding='same'))
Model.add(Conv2D(filters=512, kernel_size=3, strides=1, padding='same', activation='relu'))
Model.add(GlobalAveragePooling2D())
Model.add(Flatten())

# Fully Connected Layer
Model.add(Dense(128, activation='relu'))
Model.add(Dense(9, activation='softmax'))

# Print model summary
print(Model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 250, 250, 64)      1792      
_________________________________________________________________
average_pooling2d (AveragePo (None, 125, 125, 64)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 125, 125, 128)     73856     
_________________________________________________________________
average_pooling2d_1 (Average (None, 63, 63, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 63, 63, 512)       590336    
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 512)              

In [86]:
# Compile model

Model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='sparse_categorical_crossentropy', 
              metrics=['acc'])

In [87]:
H = model.fit(
    x_train, 
    Y_train, 
    batch_size=32, 
    epochs=100, 
    validation_data=(x_val, Y_val)
    )

Epoch 1/100
16/16 [==============================] - 33s 129ms/step - loss: nan - acc: 0.0062 - val_loss: nan - val_acc: 0.0043
Epoch 2/100
16/16 [==============================] - 1s 81ms/step - loss: nan - acc: 0.0062 - val_loss: nan - val_acc: 0.0043
Epoch 3/100
16/16 [==============================] - 1s 80ms/step - loss: nan - acc: 0.0062 - val_loss: nan - val_acc: 0.0043
Epoch 4/100
16/16 [==============================] - 1s 82ms/step - loss: nan - acc: 0.0062 - val_loss: nan - val_acc: 0.0043
Epoch 5/100
16/16 [==============================] - 1s 81ms/step - loss: nan - acc: 0.0062 - val_loss: nan - val_acc: 0.0043
Epoch 6/100
16/16 [==============================] - 1s 83ms/step - loss: nan - acc: 0.0062 - val_loss: nan - val_acc: 0.0043
Epoch 7/100
16/16 [==============================] - 1s 79ms/step - loss: nan - acc: 0.0062 - val_loss: nan - val_acc: 0.0043
Epoch 8/100
16/16 [==============================] - 1s 81ms/step - loss: nan - acc: 0.0062 - val_loss: nan - val_ac

In [88]:
H = Model.fit(
    x_train, 
    Y_train, 
    batch_size=32, 
    epochs=100, 
    validation_data=(x_val, Y_val)
    )

Epoch 1/100
16/16 [==============================] - 13s 462ms/step - loss: nan - acc: 0.0062 - val_loss: nan - val_acc: 0.0043
Epoch 2/100
16/16 [==============================] - 5s 303ms/step - loss: nan - acc: 0.0062 - val_loss: nan - val_acc: 0.0043
Epoch 3/100
16/16 [==============================] - 5s 305ms/step - loss: nan - acc: 0.0062 - val_loss: nan - val_acc: 0.0043
Epoch 4/100
16/16 [==============================] - 5s 301ms/step - loss: nan - acc: 0.0062 - val_loss: nan - val_acc: 0.0043
Epoch 5/100
16/16 [==============================] - 5s 304ms/step - loss: nan - acc: 0.0062 - val_loss: nan - val_acc: 0.0043
Epoch 6/100
16/16 [==============================] - 5s 304ms/step - loss: nan - acc: 0.0062 - val_loss: nan - val_acc: 0.0043
Epoch 7/100
16/16 [==============================] - 5s 303ms/step - loss: nan - acc: 0.0062 - val_loss: nan - val_acc: 0.0043
Epoch 8/100
16/16 [==============================] - 5s 302ms/step - loss: nan - acc: 0.0062 - val_loss: nan -